### how to use the hyperparam_opt 
to optimize hyperparameters

In [40]:
import geopandas as gp
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

import optuna
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np
import sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score

from hyperparam_opt import *
import os

In [41]:
arr = os.listdir('NASA/')
arr[1]

'POWER_Point_Monthly_Timeseries_1981_2020_046d0562N_118d3476W_LST.json'

In [42]:
os.listdir('../../hyperparameters/')

[]

In [43]:
for k in range(len(arr)):
    arr[k] = 'NASA/'+ arr[k]

In [44]:
arr.remove('NASA/POWER_Point_Monthly_Timeseries_1981_2020_047d6155N_122d2917W_LST.csv')
arr.remove('NASA/POWER_Point_Monthly_Timeseries_1981_2020_047d6155N_122d2917W_LST.xlsx')
arr.remove('NASA/POWER_Point_Monthly_Timeseries_1981_2020_045d4969N_122d5938W_LST.json')

In [45]:
arr

['NASA/POWER_Point_Monthly_Timeseries_1981_2020_046d0562N_118d3476W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_046d1514N_122d8191W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_046d1704N_123d7804W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_046d2199N_119d0837W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_046d4134N_117d0293W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_047d0347N_122d8630W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_047d1095N_119d2320W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_047d2402N_122d4345W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_047d3855N_122d1983W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_047d4189N_120d3032W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_047d5053N_121d8406W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_047d5590N_122d7195W_LST.json',
 'NASA/POWER_Point_Monthly_Timeseries_1981_2020_047d

In [46]:
arr[0]

'NASA/POWER_Point_Monthly_Timeseries_1981_2020_046d0562N_118d3476W_LST.json'

In [47]:
arr[1]

'NASA/POWER_Point_Monthly_Timeseries_1981_2020_046d1514N_122d8191W_LST.json'

In [49]:
loadjson(arr[0])

,ALLSKY_KT
198401,0.48
198402,0.46
198403,0.53
198404,0.55
198405,0.55
...,...
202009,0.57
202010,0.59
202011,0.48
202012,0.38


In [50]:
loadjson(arr[1])

,ALLSKY_KT
198401,0.48
198402,0.37
198403,0.51
198404,0.49
198405,0.52
...,...
202009,0.50
202010,0.50
202011,0.38
202012,0.38


In [51]:
for k in range(len(arr)):
#pick the location of the geojson
    filename = arr[k]

# this section is to prepare the data
    df = loadjson(filename)
    df = rm13(df)
    df = Prophet_preproc(df)

# extract the train, validation, and test data using time series splits
    X_dict, y_dict = TSS(df)

# extract just the tain and validation data
    y_train = y_dict['y_train']
    y_val = y_dict['y_val']

#Use this code to optimize the hyperparameters
    argfunction = lambda trial: objective(trial,y_train,y_val)
    study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')
    study.optimize(argfunction, n_trials=50)

    results = study.trials_dataframe()

#load the results into a csv in the location of your choosing
    results.to_csv('../../hyperparameters/SARIMAX_hyperparameter_results_' + str(k) + '.csv')

[I 2022-03-09 21:07:26,285] A new study created in memory with name: no-name-5259ea0d-7e15-48ac-9239-b049960efd2f


Optimization terminated successfully.
         Current function value: 0.677253
         Iterations: 28
         Function evaluations: 5773


[I 2022-03-09 21:08:14,838] Trial 0 finished with value: -6.806476641778367e+62 and parameters: {'p': 8, 'd': 0, 'q': 4, 'P': 2, 'D': 1, 'Q': 1}. Best is trial 0 with value: -6.806476641778367e+62.


Optimization terminated successfully.
         Current function value: -1.523258
         Iterations: 8
         Function evaluations: 2121


[I 2022-03-09 21:08:35,972] Trial 1 finished with value: 0.8322858373263375 and parameters: {'p': 4, 'd': 1, 'q': 8, 'P': 3, 'D': 0, 'Q': 3}. Best is trial 1 with value: 0.8322858373263375.
[I 2022-03-09 21:08:38,556] Trial 2 finished with value: 0.8411712622359263 and parameters: {'p': 0, 'd': 0, 'q': 0, 'P': 2, 'D': 1, 'Q': 3}. Best is trial 2 with value: 0.8411712622359263.


...

[I 2022-03-10 01:33:38,989] Trial 47 finished with value: 0.6765491997522104 and parameters: {'p': 7, 'd': 1, 'q': 4, 'P': 2, 'D': 1, 'Q': 3}. Best is trial 30 with value: 0.6810553509059212.


Optimization terminated successfully.
         Current function value: -1.310454
         Iterations: 13
         Function evaluations: 2759


[I 2022-03-10 01:34:12,164] Trial 48 finished with value: 0.6765491997522104 and parameters: {'p': 7, 'd': 1, 'q': 4, 'P': 2, 'D': 1, 'Q': 3}. Best is trial 30 with value: 0.6810553509059212.


Optimization terminated successfully.
         Current function value: -1.310454
         Iterations: 13
         Function evaluations: 2759


[I 2022-03-10 01:34:45,064] Trial 49 finished with value: 0.6765491997522104 and parameters: {'p': 7, 'd': 1, 'q': 4, 'P': 2, 'D': 1, 'Q': 3}. Best is trial 30 with value: 0.6810553509059212.


[I 2022-03-08 21:28:47,673] A new study created in memory with name: no-name-f961499b-0cd2-41a5-84e6-651fdab2abed


Optimization terminated successfully.
         Current function value: -1.451767
         Iterations: 27
         Function evaluations: 4916


[I 2022-03-08 21:29:18,289] Trial 0 finished with value: 0.7357497900655356 and parameters: {'p': 6, 'd': 3, 'q': 5, 'P': 2, 'D': 0, 'Q': 1}. Best is trial 0 with value: 0.7357497900655356.


Optimization terminated successfully.
         Current function value: -1.483059
         Iterations: 4
         Function evaluations: 508


[I 2022-03-08 21:29:24,681] Trial 1 finished with value: 0.8363155474276592 and parameters: {'p': 0, 'd': 1, 'q': 4, 'P': 1, 'D': 1, 'Q': 3}. Best is trial 1 with value: 0.8363155474276592.
[I 2022-03-08 21:29:27,797] Trial 2 finished with value: 0.14518864682078458 and parameters: {'p': 1, 'd': 0, 'q': 5, 'P': 0, 'D': 0, 'Q': 1}. Best is trial 1 with value: 0.8363155474276592.


...

[I 2022-03-08 21:42:14,864] Trial 47 finished with value: 0.8362010317360907 and parameters: {'p': 0, 'd': 0, 'q': 2, 'P': 3, 'D': 1, 'Q': 2}. Best is trial 14 with value: 0.8403786535811981.


Optimization terminated successfully.
         Current function value: -1.541758
         Iterations: 9
         Function evaluations: 1575


[I 2022-03-08 21:42:27,157] Trial 48 finished with value: 0.8393076205914691 and parameters: {'p': 2, 'd': 0, 'q': 5, 'P': 2, 'D': 1, 'Q': 2}. Best is trial 14 with value: 0.8403786535811981.


Optimization terminated successfully.
         Current function value: -1.472806
         Iterations: 15
         Function evaluations: 2780


[I 2022-03-08 21:42:59,454] Trial 49 finished with value: 0.8347209765328292 and parameters: {'p': 7, 'd': 1, 'q': 1, 'P': 3, 'D': 1, 'Q': 2}. Best is trial 14 with value: 0.8403786535811981.
